# 三角套利
[链接](https://zhuanlan.zhihu.com/p/44428321)

![](note_pic/v2-284b46480f0e8b50e58a69d62b92f0b7_1440w.jpg)

## 正向套利
从图中可知：
$$
\begin{cases}
\frac{eos}{BTC} = P_1 \\
\frac{BTC}{USDT} = P_2 \\
\frac{eos}{USDT} = P_3 \\
\end{cases}
$$
其中，p1、p2为买入价，p3为卖出价

从上方程组可知：
$$
\begin{cases}
\frac{eos}{BTC} = P_1 \\
\frac{BTC}{USDT} = P_2 \\
\end{cases} \Rightarrow \frac{eos}{USDT} = P_1 P_2
$$
即买入一eos，需要花费$P_1P_2$个USDT。当P3等于P1 * P2时，无套利机会

当$P_1P_2 < P_3$时，说明cost（购入1eos花费的P1 * P2个USDT）小于payoff（卖出1eos获得的P3个USDT），profit = (p3 - p1 * p2)个usdt

此情况被称为正向套利

## 反向套利
即上述的相反情况，$P_1P_2 > P_3$。套利过程为：
1. 卖出1个eos，获得p1个btc
2. 将上一步得到的p1个btc全部卖出，获得p1 * p2个usdt
3. 从图中可知，如果直接用usdt买入eos，汇率为1 eos = p3 usdt
4. 平掉eos空仓，如果获得的usdt（p1 * p2）大于花费的usdt（p3），则获利

In [6]:
import ccxt
import pandas as pd
import time

pd.set_option('expand_frame_repr', False)

In [7]:
# 初始化交易所
okex_exchange = ccxt.okex({
    'Timeout': 15000,
    'enableRateLimit': True
})

# 加载行情
markets = okex_exchange.load_markets()

In [8]:
# == Step.1 选择两个交易市场 A, B
market_a = 'BTC'
market_b = 'ETH'
# == Step.1 END =================

In [9]:
# == Step.2 找到所有同时以 A 和 B 都作为计价的货币
# 市场内的交易对
symbols = list(markets.keys())

# 存放到DataFrame中
symbols_df = pd.DataFrame(data=symbols, columns=['symbol'])

# 分割字符串得到 基础货币/计价货币
base_quote_df = symbols_df['symbol'].str.split(pat='/', expand=True)
base_quote_df.columns = ['base', 'quote']

# 过滤得到以 A, B 计价的计价货币
base_a_list = base_quote_df[base_quote_df['quote'] == market_a]['base'].values.tolist()
base_b_list = base_quote_df[base_quote_df['quote'] == market_b]['base'].values.tolist()

# 获取相同的基础货币列表
common_base_list = list(set(base_a_list).intersection(set(base_b_list))) # intersection:获得多个参数中相同的部分
# print('{} and {} have {}个相同的计价货币'.format(market_a, market_b, len(common_base_list)))
# == Step.2 END =================

In [27]:
# == Step.3 执行套利步骤

# 结果保存到DataFrame中
columns = ['Market A',
           'Market B',
           'Market C',
           'P1',
           'P2',
           'P3',
           'Profit in %']

results_df = pd.DataFrame(columns=columns)

# 获取前一分钟的close价格
last_min = okex_exchange.milliseconds() - 60 * 1000  # 前一分钟

for base_coin in common_base_list:
    market_c = base_coin
    market_a2b_symbol = '{}/{}'.format(market_b, market_a)
    market_b2c_symbol = '{}/{}'.format(market_c, market_b)
    market_a2c_symbol = '{}/{}'.format(market_c, market_a)

    # 获取行情前一分钟的K线数据(这里无法获取当前一分钟的K线数据，日后检查）
    market_a2b_kline = okex_exchange.fetch_ohlcv(market_a2b_symbol, since=last_min, limit=1, timeframe='1m')
    market_b2c_kline = okex_exchange.fetch_ohlcv(market_b2c_symbol, since=last_min, limit=1, timeframe='1m')
    market_a2c_kline = okex_exchange.fetch_ohlcv(market_a2c_symbol, since=last_min, limit=1, timeframe='1m')

    if len(market_a2b_kline) == 0 or len(market_b2c_kline) == 0 or market_a2c_kline == 0:
        print(f'{market_c}:no history data received')
        print('-'*64)
        pass
    else:
        p1 = market_a2b_kline[0][4]
        p2 = market_b2c_kline[0][4]
        p3 = market_a2c_kline[0][4]

        # 价差
        profit = (p3 / (p1 * p2) - 1) * 1000
        '''
        p1 = eth/btc
        p2 = x/eth
        p3 = x/btc
        '''
        results_df = results_df.append({
            'Market A': market_a,
            'Market B': market_b,
            'Market C': market_c,
            'P1': p1,
            'P2': p2,
            'P3': p3,
            'Profit in %': profit
        }, ignore_index=True)

        # 显示信息
        print(results_df.tail(1))
        print('-'*64)

        # # 防止超过rate limit
        # time.sleep(okex_exchange.rateLimit / 1000)

NULS:no history data received
----------------------------------------------------------------
XLM:no history data received
----------------------------------------------------------------
YFI:no history data received
----------------------------------------------------------------
QTUM:no history data received
----------------------------------------------------------------
MKR:no history data received
----------------------------------------------------------------
LTC:no history data received
----------------------------------------------------------------
OKT:no history data received
----------------------------------------------------------------
AAVE:no history data received
----------------------------------------------------------------
ETC:no history data received
----------------------------------------------------------------
KSM:no history data received
----------------------------------------------------------------
FLOW:no history data received
---------------------------